<a href="https://colab.research.google.com/github/siroj05/Tugas-Machine-Learning/blob/main/Week%207/Distributed_XGBoost_with_Ray.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**Installing and starting Ray**



In [ ]:
pip install ray

     |████████████████████████████████| 54.7 MB 31 kB/s 
     |████████████████████████████████| 72 kB 481 kB/s 


**Installing XGBoost-Ray**

In [ ]:
pip install xgboost_ray

     |████████████████████████████████| 136 kB 13.6 MB/s 


**Installing SKLearn**

In [ ]:
pip install sklearn

**Simple training example**

In [ ]:
from xgboost_ray import RayDMatrix, RayParams, train
from sklearn.datasets import load_breast_cancer

train_x, train_y = load_breast_cancer(return_X_y=True)
train_set = RayDMatrix(train_x, train_y)

evals_result = {}
bst = train(
    {
        "objective": "binary:logistic",
        "eval_metric": ["logloss", "error"],
    },
    train_set,
    evals_result=evals_result,
    evals=[(train_set, "train")],
    verbose_eval=False,
    ray_params=RayParams(num_actors=2, cpus_per_actor=1))

bst.save_model("model.xgb")
print("Final training error: {:.4f}".format(
    evals_result["train"]["error"][-1]))

/usr/local/lib/python3.7/dist-packages/xgboost_ray/main.py:166: UserWarning: You are using `xgboost_ray` with a legacy XGBoost version (version 0.90). While we try to support older XGBoost versions, please note that this library is only fully tested and supported for XGBoost >= 1.4. Please consider upgrading your XGBoost version (`pip install -U xgboost`).
  warnings.warn(LEGACY_WARNING)
2021-11-10 17:50:13,618	INFO main.py:971 -- [RayXGBoost] Created 2 new actors (2 total actors). Waiting until actors are ready for training.
2021-11-10 17:50:15,245	INFO main.py:1016 -- [RayXGBoost] Starting XGBoost training.
(pid=292) [17:50:15] WARNING: /workspace/src/learner.cc:622: Tree method is automatically selected to be 'approx' for distributed training.
(pid=292) [17:50:15] Tree method is automatically selected to be 'approx' for distributed training.
(pid=316) [17:50:15] WARNING: /workspace/src/learner.cc:622: Tree method is automatically selected to be 'approx' for distributed training.
(pi

Final training error: 0.0053


**Simple prediction example**

In [ ]:
from xgboost_ray import RayDMatrix, RayParams, predict
from sklearn.datasets import load_breast_cancer
import xgboost as xgb

data, labels = load_breast_cancer(return_X_y=True)

dpred = RayDMatrix(data, labels)

bst = xgb.Booster(model_file="model.xgb")
pred_ray = predict(bst, dpred, ray_params=RayParams(num_actors=2))

print(pred_ray)

/usr/local/lib/python3.7/dist-packages/xgboost_ray/main.py:166: UserWarning: You are using `xgboost_ray` with a legacy XGBoost version (version 0.90). While we try to support older XGBoost versions, please note that this library is only fully tested and supported for XGBoost >= 1.4. Please consider upgrading your XGBoost version (`pip install -U xgboost`).
  warnings.warn(LEGACY_WARNING)
2021-11-10 17:50:43,743	INFO main.py:1535 -- [RayXGBoost] Created 2 remote actors.
2021-11-10 17:50:45,360	INFO main.py:1552 -- [RayXGBoost] Starting XGBoost prediction.


[0.07511145 0.04715708 0.02642454 0.11464816 0.08779665 0.19609766
 0.02642454 0.03200296 0.03968306 0.06930447 0.09290799 0.02642454
 0.03476055 0.05490047 0.05012454 0.02642454 0.03337562 0.02642454
 0.02642454 0.9528658  0.9751126  0.9751126  0.07511145 0.02642454
 0.02642454 0.04102892 0.03118352 0.02642454 0.02642454 0.08779665
 0.02642454 0.03476055 0.02642454 0.02642454 0.02642454 0.02642454
 0.0665335  0.9690314  0.33619487 0.0696831  0.5729184  0.15662025
 0.03451282 0.03200296 0.12301622 0.04102892 0.9751126  0.04096693
 0.97315884 0.9234187  0.9751126  0.9751126  0.9751126  0.03059936
 0.09855346 0.9751126  0.02642454 0.03755146 0.9751126  0.97315884
 0.9751126  0.97315884 0.02642454 0.9751126  0.03120249 0.03120249
 0.97315884 0.9751126  0.8630876  0.9751126  0.02642454 0.9751126
 0.02642454 0.18486539 0.9751126  0.02642454 0.9684864  0.07511145
 0.04528174 0.9751126  0.97315884 0.8423582  0.02642454 0.02642454
 0.9751126  0.02642454 0.18525673 0.02642454 0.9610139  0.91992

**Hyperparameter optimization**

In [ ]:
from xgboost_ray import RayDMatrix, RayParams, train
from sklearn.datasets import load_breast_cancer

num_actors = 1
num_cpus_per_actor = 1

ray_params = RayParams(
    num_actors=num_actors, cpus_per_actor=num_cpus_per_actor)

def train_model(config):
    train_x, train_y = load_breast_cancer(return_X_y=True)
    train_set = RayDMatrix(train_x, train_y)

    evals_result = {}
    bst = train(
        params=config,
        dtrain=train_set,
        evals_result=evals_result,
        evals=[(train_set, "train")],
        verbose_eval=False,
        ray_params=ray_params)
    bst.save_model("model.xgb")

from ray import tune

# Specify the hyperparameter search space.
config = {
    "tree_method": "approx",
    "objective": "binary:logistic",
    "eval_metric": ["logloss", "error"],
    "eta": tune.loguniform(1e-4, 1e-1),
    "subsample": tune.uniform(0.5, 1.0),
    "max_depth": tune.randint(1, 9)
}

# Make sure to use the `get_tune_resources` method to set the `resources_per_trial`
analysis = tune.run(
    train_model,
    config=config,
    metric="train-error",
    mode="min",
    num_samples=4,
    resources_per_trial=ray_params.get_tune_resources())
print("Best hyperparameters", analysis.best_config)

2021-11-10 17:51:05,398	WARNING function_runner.py:564 -- Function checkpointing is disabled. This may result in unexpected behavior when using checkpointing features or certain schedulers. To enable, set the train function arguments to be `func(config, checkpoint_dir=None)`.
2021-11-10 17:51:05,414	INFO logger.py:606 -- pip install 'ray[tune]' to see TensorBoard files.
2021-11-10 17:51:05,415	WARNING callback.py:115 -- The TensorboardX logger cannot be instantiated because either TensorboardX or one of it's dependencies is not installed. Please make sure you have the latest version of TensorboardX installed: `pip install -U tensorboardx`


Trial name,status,loc,eta,max_depth,subsample
train_model_c6efe_00000,PENDING,,0.0033019,5,0.501565
train_model_c6efe_00001,PENDING,,0.00107545,2,0.853439
train_model_c6efe_00002,PENDING,,0.000105926,6,0.920638
train_model_c6efe_00003,PENDING,,0.0390703,1,0.637116


(pid=406) /usr/local/lib/python3.7/dist-packages/xgboost_ray/main.py:166: UserWarning: You are using `xgboost_ray` with a legacy XGBoost version (version 0.90). While we try to support older XGBoost versions, please note that this library is only fully tested and supported for XGBoost >= 1.4. Please consider upgrading your XGBoost version (`pip install -U xgboost`).
(pid=406)   warnings.warn(LEGACY_WARNING)
(pid=406) /usr/local/lib/python3.7/dist-packages/xgboost_ray/main.py:422: UserWarning: `num_actors` in `ray_params` is smaller than 2 (1). XGBoost will NOT be distributed!
(pid=406)   f"`num_actors` in `ray_params` is smaller than 2 "
(pid=406) 2021-11-10 17:51:07,295	INFO main.py:971 -- [RayXGBoost] Created 1 new actors (1 total actors). Waiting until actors are ready for training.
(pid=406) 2021-11-10 17:51:09,306	INFO main.py:1016 -- [RayXGBoost] Starting XGBoost training.


Result for train_model_c6efe_00000:
  date: 2021-11-10_17-51-10
  done: false
  experiment_id: caf215c8a90f4f4884f4cfc4e3f43f82
  hostname: 2c6e832e1a8d
  iterations_since_restore: 1
  node_ip: 172.28.0.2
  pid: 406
  time_since_restore: 2.907121419906616
  time_this_iter_s: 2.907121419906616
  time_total_s: 2.907121419906616
  timestamp: 1636566670
  timesteps_since_restore: 0
  train-error: 0.057996
  train-logloss: 0.690397
  training_iteration: 1
  trial_id: c6efe_00000
  
Result for train_model_c6efe_00000:
  date: 2021-11-10_17-51-10
  done: true
  experiment_id: caf215c8a90f4f4884f4cfc4e3f43f82
  experiment_tag: 0_eta=0.0033019,max_depth=5,subsample=0.50157
  hostname: 2c6e832e1a8d
  iterations_since_restore: 10
  node_ip: 172.28.0.2
  pid: 406
  time_since_restore: 2.972598075866699
  time_this_iter_s: 0.005982875823974609
  time_total_s: 2.972598075866699
  timestamp: 1636566670
  timesteps_since_restore: 0
  train-error: 0.033392
  train-logloss: 0.666235
  training_iteration

(pid=406) 2021-11-10 17:51:10,223	INFO main.py:1498 -- [RayXGBoost] Finished XGBoost training on training data with total N=569 in 2.97 seconds (0.91 pure XGBoost training time).


...

Result for train_model_c6efe_00003:
  date: 2021-11-10_17-51-24
  done: false
  experiment_id: 2fa57007de86493db75e8829c2d12617
  hostname: 2c6e832e1a8d
  iterations_since_restore: 1
  node_ip: 172.28.0.2
  pid: 718
  time_since_restore: 2.997675657272339
  time_this_iter_s: 2.997675657272339
  time_total_s: 2.997675657272339
  timestamp: 1636566684
  timesteps_since_restore: 0
  train-error: 0.082601
  train-logloss: 0.666329
  training_iteration: 1
  trial_id: c6efe_00003
  
Result for train_model_c6efe_00003:
  date: 2021-11-10_17-51-24
  done: true
  experiment_id: 2fa57007de86493db75e8829c2d12617
  experiment_tag: 3_eta=0.03907,max_depth=1,subsample=0.63712
  hostname: 2c6e832e1a8d
  iterations_since_restore: 10
  node_ip: 172.28.0.2
  pid: 718
  time_since_restore: 3.0531466007232666
  time_this_iter_s: 0.004805326461791992
  time_total_s: 3.0531466007232666
  timestamp: 1636566684
  timesteps_since_restore: 0
  train-error: 0.063269
  train-logloss: 0.487603
  training_iteration

(pid=718) 2021-11-10 17:51:24,078	INFO main.py:1498 -- [RayXGBoost] Finished XGBoost training on training data with total N=569 in 3.05 seconds (0.90 pure XGBoost training time).


Trial name,status,loc,eta,max_depth,subsample,iter,total time (s),train-logloss,train-error
train_model_c6efe_00000,TERMINATED,172.28.0.2:406,0.0033019,5,0.501565,10,2.9726,0.666235,0.033392
train_model_c6efe_00001,TERMINATED,172.28.0.2:510,0.00107545,2,0.853439,10,2.9645,0.684501,0.036907
train_model_c6efe_00002,TERMINATED,172.28.0.2:614,0.000105926,6,0.920638,10,3.06288,0.692203,0.010545
train_model_c6efe_00003,TERMINATED,172.28.0.2:718,0.0390703,1,0.637116,10,3.05315,0.487603,0.063269


2021-11-10 17:51:24,237	INFO tune.py:630 -- Total run time: 18.84 seconds (18.66 seconds for the tuning loop).


Best hyperparameters {'tree_method': 'approx', 'objective': 'binary:logistic', 'eval_metric': ['logloss', 'error'], 'eta': 0.00010592618712694762, 'subsample': 0.9206383466746203, 'max_depth': 6}
